In [ ]:
# setup

# I use my own startup.jl file. See:
# https://gist.github.com/AntonOresten/e37c22afe5798604a43eb520e3f40c71/5c377e1a426e078ed3716c6b905a1832109b2332

using Pkg
Pkg.activate("intro", shared=true) # shared environment
Pkg.add([
    "CUDA", "Colors", "Einops", "SpecialFunctions",
    "Symbolics", "ThreadsX", "Zygote", "StyledStrings",
    "Optimisers",
])

using StyledStrings
using Dates

function welcome()
    poem = styled"""
        Roses are {red:red}
            Violets are {blue:blue}
          Welcome to my talk.
              My name is {green:Anton}.
            I'm a bioinformatician
                in the Murrell lab
              here at Karolinska.
                  I like making Julia packages.
                The time is currently $(now()) (UTC)"""

    print(poem)
end;

# ***Introduction to Scientific Programming in Julia***

<img width="25%" src="https://seekvectors.com/files/download/Julia%20Language-logo.png">

In [ ]:
welcome()

# **Why Julia?** - an intro-intro

## **The two-language problem**

<img width="70%" src="https://cdn.hashnode.com/res/hashnode/image/upload/v1681735971356/91b6e886-7ce1-41a3-9d9f-29b7b096e7f2.png" />

Programmers are often faced with a dilemma when choosing a language for a project, often having to trade accessibility and development speed for performance:

- Interpreted languages like *Python* and *R* have relatively simple APIs and syntax, and a clear separation of concerns between the user and machine. While allowing end users to do great things using pre-bundled functions, this simplicity limits lower level control over hardware and performance.

- Compiled languages like *C++* and *Rust* require more technical knowledge, and users often need to work at the hardware level, limiting the ability to interactively work with problems at a high level in tight feedback loops. These compiled languages deliver performance but require lengthy compile-edit-debug cycles.

### **Example: approximating $\pi$ with the Basel series**

The sum of the first 100 million terms in the Basel series can be expressed in mathematical terms as:

$$
\sum_{i=1}^{10^8} \frac{1}{i^2} = \frac{1}{1} + \frac{1}{4} + \frac{1}{9} + \ldots + \frac{1}{10^{16}} \approx \frac{\pi^2}{6}
$$

---
```py
# Example Python code:

def compute_series(n):
    return sum(1 / i**2 for i in range(1, n+1))

result = compute_series(100_000_000)  # takes ~23 seconds
print(result)
```
---

**Python** lets us to be concise in our implementation, but is embarrassingly slow.

We can achieve some speed-up by expanding the loop, and working at a *slightly* lower level:

---
```py
# Alternative Python code:

def compute_series(n):
    total = 0.0
    for i in range(1, n+1):
        total += 1 / (i * i)
    return total

result = compute_series(100_000_000)  # takes ~8.8 seconds
print(result)
```
---

What's the point of rewriting our clean and readable high-level code to verbose and less readable lower-level code if it's still going to be slow?

The function body and logic is starting to look like C++, which runs almost two orders of magnitude faster:

---
```cpp
// Example C++ code:

#include <iostream>
double compute_series(int n) {
    double total = 0.0;
    for (long long i = 1; i <= n; ++i) {
        total += 1.0 / (i * i);
    }
    return total;
}

int main() {
    double result = compute_series(100000000);  // takes ~0.15 seconds
    std::cout << result << std::endl;
    return 0;
}
```
---

<img width="70%" src="https://cdn.hashnode.com/res/hashnode/image/upload/v1681735992315/62fdd58f-4630-4120-8eb4-5238740543e8.png" />



In [ ]:
# Julia implementation
# as readable as Python
# already on par with, if not faster than C++

function compute_series(n)
    return sum(1 / i^2 for i in 1:n)
end

@time compute_series(100_000_000)

Julia achieves this through just-in-time compilation

- your code gets compiled to machine code the first time it runs.

- results in C-like performance with Python-like syntax.

In [ ]:
# Julia maintains its performance
# even when written out explicitly

function compute_series(n)
    s = 0.0
    for i in 1:n
        s += 1 / i^2
    end
    return s 
end

@time compute_series(100_000_000)

- This is a simple numerical calculation.
- **It should not have to take more than 5 lines of precious space.**
- Julia can do it in one:

In [ ]:
compute_series(n) = sum(1 / i^2 for i in 1:n)

@time compute_series(100_000_000)

In [ ]:
# native multi-threading support
# with convenient packages for leveraging it 

using ThreadsX

compute_series_threaded(n) = ThreadsX.sum(1 / i^2 for i in 1:n)

In [ ]:
@time compute_series_threaded(100_000_000)

<img width="30%" src="https://images-cdn.welcomesoftware.com/Zz00ZWNmM2UzZWNjOTExMWVjOTIxNWEyNmQ2ODJlMTgxMA==/Radiology%20cherry-picking%20and%20workforce%20optimization.jpeg" />

### **Am I cherry-picking?**
- **Yes. Computation is one of the things Julia excels at.**
- **What you'll find is that Julia has a disproportionate amount of very sweet cherries.**
- **The contrast is not always this stark, and language performance is a nuanced topic!**

---

## **Julia uses a multiple dispatch, functional programming paradigm** 

### 1. A single *function* can have multiple *methods*

"multiple" includes none at all:

```julia
    # a function with no methods
    function foo end
```

`foo` is just a *name* to which we can add methods.

Functions are extended with new methods to accommodate different types under one umbrella to support both concrete and abstract types.

<img src="https://github.com/user-attachments/assets/192dd9e1-ca2f-4771-9fe1-92da5b8eb8f1" />

In [ ]:
foo(x::Integer) = 1        # foo(5) or foo(0x05)
foo(x::AbstractFloat) = 2  # foo(5.0) or foo(5.0f0)

foo(5)

In [ ]:

foo(5.0)

In [ ]:
foo(x::Bool) = 3

foo(true)

### 2. Dispatching is based on the types of *all* arguments

```julia
    foo(x::Real, y::Real) = x + y
```

Methods do *not* just belong to types themselves (as opposed to e.g. Python-style single dispatch).

```julia
    foo(x::Array{<:Real}, y::Real) = sum(x) + y  # foo([1, 2], 3)
    foo(x::Real, y::Array{<:Real}) = x + sum(y)  # foo(2, [3, 4])
```

Method ambiguities (like `foo([1], [2])`) do not get resolved, and require disambiguation either :

- defining `foo(::Array{<:Real}, ::Array{<:Real})`



### 3. The compiler specializes on each combination of argument types

In [ ]:
pow(x::Real, y::Real) = x^y

In [ ]:
pow(2, 3)

In [ ]:
pow(2.0, 3.0)

In [ ]:
pow(1, 2.0)

In [ ]:
methods(^)

In [ ]:
@code_llvm pow(2, 3)

In [ ]:
@code_llvm pow(2, 3.0)

- Julia will compile `pow` three times
  1. `x::Int, y::Int`
  2. `x::Float64, y::Float64`
  3. `x::Int, y::Float64`, which includes converting `x` to a `Float64`

- By compiling different specializations Julia can do all the method lookup ahead of time.

- Julia specializes on the *concrete* type.

- Method signatures are just constraints for dispatching.

### 4. Methods can have additional constraints beyond individual argument types

<img width="80%" src="https://cdn.hashnode.com/res/hashnode/image/upload/v1681980418289/8da099db-a243-4908-8a55-b4e2a999fc0d.png?auto=compress,format&format=webp" />

In [ ]:
# general case
same_type(x, y) = false

# case where types of x and y match
same_type(x::T, y::T) where {T} = true

In [ ]:
same_type(1, "hello")

In [ ]:
same_type(1, 2)

## **Metaprogramming**

As you have noticed, I've been using `@time` to time the execution of function calls.
These are not just magic words, but are functions that change the input expressions. 

- Metaprogramming is about using programming to write code itself.
- It's a powerful tool that lets you abstract away boilerplate code by hiding it behind a **macro**.

In [ ]:
@time sleep(1)

In [ ]:
@assert 1 == 2

In [ ]:
@macroexpand @assert 1 == 2

## **`Array`, `AbstractArray`, and beyond**

For scientific computation, arrays are very important,
and with different representations of arrays, Julia's multiple dispatch paradigm proves useful

### The `Array` type

In Julia, the `Array` type is a densely stored collection of elements, with a dimensionality and shape.

When the dimensionality is `1`, we call the array a `Vector`:

In [ ]:
v = [1, 2, 3]

In [ ]:
v isa Vector{Int}

In [ ]:
Vector

In [ ]:
v isa Array

In [ ]:
# it can be mutated to add more elements
push!(v, 4)

In [ ]:
deleteat!(v, 4)

In [ ]:
append!(v, 4:6)

In [ ]:
deleteat!(v, 4:6)

In [ ]:
# vectors in Julia are technically column vectors
# row vectors are matrices of size (1, length)
[1 2 3 4]

In [ ]:
m = [1 2 3;
     4 5 6]

In [ ]:
# transposing (swapping first and second dimension)
m'

In [ ]:
ones(2, 3)

In [ ]:
reshape(fill(100.0, (4, 10)), 4, 5, 2)

In [ ]:
A = reshape(1:6, 3, 2)'

In [ ]:
B = reshape(7:12, 2, 3)'

In [ ]:
# matrix multiplications are commonly used
A * B

<img width="70%" src="https://www.mathsisfun.com/algebra/images/matrix-multiply-a.svg" />

### **The `AbstractArray{T,N}` type**

- An abstract type, parameterized with element type and array dimensionality.

- Agnostic to storage patterns and devices (CPU, GPU).

- Implements generic definitions for any array, e.g.:

  - `length(array::AbstractArray)` is `prod(size(array))`.

  - `sum(array::AbstractArray)` iterates over elements and returns the sum.

  - `reshape(array::AbstractArray, size)` .
  
    - `Array` just holds a pointer and size, so `reshape(::Array, size)` keeps the pointer, but changes the `size`

In [ ]:
using LinearAlgebra

# a sparsely stored AbstractMatrix type
sparse_diagonal = Diagonal(rand(1:4, 100))

In [ ]:
@be sum($sparse_diagonal)

In [ ]:
dense_diagonal = collect(sparse_diagonal)

In [ ]:
@be sum($dense_diagonal)

In [56]:
"""
    OneHotArray{N}

Stores the index and size of an array, with the element at the given index being `true`,
and all other elements being `false`.
"""
struct OneHotArray{N} <: AbstractArray{Bool,N}
    index::NTuple{N,Int}
    size::NTuple{N,Int}
end

Base.size(A::OneHotArray) = A.size

Base.getindex(A::OneHotArray{N}, i::Vararg{Int,N}) where {N} = (i == A.index)

OneHotArray(index::Int, length::Int) = OneHotArray((index,), (length,))

onehot(element, collection::AbstractArray) = OneHotArray(Tuple(findfirst(==(element), collection)), size(collection))
onehot(element, collection) = OneHotArray(findfirst(==(element), collection), length(collection))

unhot(A::OneHotArray, collection) = collection[A.index...]

# pretty printing with ⋅ for false values
function Base.replace_in_print_matrix(x::OneHotArray{N}, i::Integer, j::Integer, s::AbstractString) where N
    x[i,j] ? s : Base.replace_with_centered_mark(s)
end


In [ ]:
# a constructor has been implicitly created:
A = OneHotArray((3, 4), (6, 8))

In [ ]:
B = OneHotArray(50_000, 100_000)

@be sum($B)

In [59]:
Base.sum(A::OneHotArray) = 1

In [ ]:
@be sum($B)

In [ ]:
A = rand(30, 40)

In [ ]:
B = OneHotArray(2, 40)

In [ ]:
@be $A * $B

In [ ]:
import Base: *

const OneHotVector = OneHotArray{1}
const OneHotMatrix = OneHotArray{2}

(*)(A::AbstractMatrix, B::OneHotVector) = A[:, B.index...]

In [ ]:
@be $A * $B

In [ ]:
(*)(A::AbstractMatrix, B::OneHotVector) = @view A[:, B.index...]

In [ ]:
@be $A * $B

## **Broadcasting and Vectorization**

Loop-free, allocation-free, element-wise arithmetic

* **What it is:**
    - A *dot* (`.`) placed in front of a function or operator tells Julia to call that
      function on **each element** of its arguments.  
    - The compiler fuses every dotted operation in the same expression into a
      single loop, so you pay for *one* pass and no intermediate arrays.

*   **Why it matters:**
    - Speed comparable to a handwritten for loops.
    - Works the same on CPU and GPU arrays.  
    - Plays nicely with broadcasting assignment (`.=`) so you can update
      arrays in-place.

### 1. Basic element-wise arithmetic

In [ ]:
a = collect(1:5)

In [ ]:
b = collect(1:5) .* 10

In [ ]:
c = a .+ b

In [ ]:
d = sin.(a) .+ cos.(b)

In [ ]:
e = @. exp(a) / (1 + b)    # @. adds the dot *everywhere*

### 2. In-place updates with `.=`

In [ ]:
x = rand(10^6)
y = rand(10^6)

x .+= 2y .- 0.5            # mutate `x` in one fused operation, no allocations

### 3. Functions broadcast for free

In [ ]:
hypot(a, b) = sqrt(a^2 + b^2)

a  = 3
b  = [1, 2, 3, 4, 5]

c = hypot.(a, b)       # 2-D broadcast (row × column) → Matrix

In [86]:
function loop_add!(dest, p, q)
    @inbounds for i in eachindex(dest)
        dest[i] = p[i] + q[i]
    end
    dest
end

p = rand(10^6)
q = rand(10^6)
dest = similar(p);

In [ ]:
@be loop_add!(dest, p, q)  # handwritten loop

In [ ]:
@be dest .= p .+ q         # fused broadcast

### 4. Broadcasting will repeat "singleton" dimensions

<img width="30%" src="https://cdn.hashnode.com/res/hashnode/image/upload/v1695327323207/2mac9kM1v.png?auto=format" />

In [ ]:
a = [1 2 3]

In [ ]:
b = reshape(1:18, 6, 3)

In [ ]:
a .* b

In [ ]:
# equivalent to single line, comma-separated:
v = [2
     3
     5
     7]


In [ ]:
reshape(v, 1, :)

In [ ]:
# A (size m×1) and B (1×n
v .* v'

### **Parallelism**

- Hardware is becoming increasingly parallel.
- Languages like Python suffer from design choices (like the GIL) that prevent you from leveraging CPU-bound multi-threading natively.
- Julia code natively extends to parallel hardware.

In [97]:
A = rand(100_000_000);

In [ ]:
polynomial(x) = @. x + 2x^2 - x^3

In [ ]:
@be polynomial($A)

In [ ]:

B = cu(A)

In [ ]:
using CUDA


In [ ]:
@be CUDA.@sync polynomial(B)

### **Einops**

<center>
  <img width="80%" alt="Image" src="https://github.com/user-attachments/assets/73d61b66-90ef-4927-add0-c118cf973626" align="middle" />
</center>

In [ ]:
using Einops, Colors

image = [colorant"red" colorant"lime" colorant"blue"
         colorant"orange" colorant"cyan" colorant"magenta"]

In [ ]:
size(image)

In [ ]:
reshape(image, 1, 6)

In [ ]:
rearrange(image, einops"h w -> w h")

In [ ]:
repeat(image, einops"h w -> (h 2) (w 2)")

In [ ]:
tiled_image = repeat(image, einops"h w -> (2 h 3) (4 w)")

In [ ]:
A = reshape(collect(1:30), 3, 5, 2)

In [ ]:
rearrange(A, einops"h w b -> w h b")

## **Automatic differentiation (AD)**



Derivatives/gradients compute the rate of change of a function's output with respect to input(s)

$$
\frac{d}{dx}f(x) = \lim_{h \rightarrow 0} \frac{f(x + h) - f(x)}{h}
$$

- Optimization problems
- Machine learning


In [ ]:
# automatic differentiation package
using Zygote

h(x) = sin(x)^2
h′(x) = gradient(h, x)

In [ ]:
n = 2.0
@be h′($n)

### **AD backends like Zygote scale to gradients of very big and complex functions with millions/billions of learnable parameters**

In [105]:
struct Dense
    weight::AbstractMatrix
    bias::AbstractVector
    activation::Function
end

function Dense(in_dim::Integer, out_dim::Integer, activation=identity)
    weight = randn(out_dim, in_dim) * sqrt(2 / (in_dim+out_dim))
    bias = zeros(out_dim)
    Dense(weight, bias, activation)
end

(dense::Dense)(x::AbstractVecOrMat) = dense.activation.(dense.weight * x .+ dense.bias)

function (dense::Dense)(x::AbstractArray)
    x_flat = reshape(x, size(x, 1), :)
    y_flat = dense(x_flat)
    y = reshape(y_flat, size(y_flat, 1), size(x)[2:end]...)
    return y
end

In [ ]:
using Zygote, Statistics

model = Dense(8, 2)

n_batches = 10
x = rand(8, n_batches)
y = [zeros(n_batches ÷ 2); ones(n_batches ÷ 2)]'

losses = []
learning_rate = 0.01
for i in 0:1000
    loss, (∇model,) = Zygote.withgradient(model -> mean(abs2, model(x) .- y), model)
    model.weight .-= ∇model.weight * learning_rate
    model.bias .-= ∇model.bias * learning_rate

    push!(losses, loss)
    iszero(i % 100) && println("i = $i, loss = $loss")
end

### Bonus: Symbolic Differentiation

In [ ]:
using Symbolics, SpecialFunctions

@variables x y

z = x^2 * sin(cbrt(y)*x)^x^gamma(gamma(x))

In [ ]:
Symbolics.derivative(z, x)

In [ ]:
dz_dx = Symbolics.derivative(z, x)
f′ = eval(build_function(dz_dx, x, y))

In [ ]:
@be f′($1, $2)

## **Ecosystem and interoperability**

- Julia has a rich ecosystem with >10000 packages.
- There exists thousands of packages providing bindings for compiled binaries.


### **Compile-time latency**

- Julia suffers from high "Time To First X" (TTFX).
- Running methods for the first time after updating your packages can have high latency depending on dependencies.

### **Calling C libraries from Julia**

In [ ]:
run(`gcc -shared -fPIC -O3 -o compute_series.so examples/compute_series.c`)

In [ ]:
function compute_series_c(n::Int)
    ccall((:compute_series, "./compute_series.so"), Cdouble, (Cint,), n)
end

In [ ]:
@time compute_series_c(100_000_000)

## **Installation and setup**

The recommended way of installing Julia is using `juliaup`:

### Mac, Linux:

`curl -fsSL https://install.julialang.org | sh`

### Windows:

`winget install --name Julia --id 9NJNWW8PVKMN -e -s msstore`

### Make your Julia experience smoother with a `startup.jl` file:

- Runs every time you start Julia.
- Optionally disable with `julia --startup-file=no`.
- Located at `~/.julia/config/startup.jl`

In [ ]:
@bs sin(1)

In [ ]:
# package/code updates through file-watching
using Revise

# `{@b,@be,@bs} <expr>` for fast benchmarking
using PrettyChairmarks

# syntax highlighting, bracket completion
using OhMyREPL

# colorschemes at:
# https://kristofferc.github.io/OhMyREPL.jl/stable/features/syntax_highlighting/
colorscheme!("GitHubDark")

# fix bug where `[` + `]` results in `[]]`
@async begin
    while !isdefined(Base, :active_repl) sleep(0.1) end
    OhMyREPL.Prompt.insert_keybindings()
end

# ^^ this snippet is a subset of https://gist.github.com/AntonOresten/e37c22afe5798604a43eb520e3f40c71

### AntonOresten on GitHub

This notebook will be available at https://github.com/AntonOresten/Intro.jl

<img width="20%" src="https://avatars.githubusercontent.com/u/96627903?v=4" />

### If you have any questions you could:
- email me at antonoresten@gmail.com
- open an issue in https://github.com/AntonOresten/Intro.jl
- ask ChatGPT, Claude, Gemini, or DeepSeek, etc.
- do NOT ask `hotdog-bruh-v1`

![Image](https://github.com/user-attachments/assets/2b59c01b-94b5-4642-bf00-7b3423d35806)

![Image](https://github.com/user-attachments/assets/dda22aac-5958-439f-92c5-2b34480a605a)
